In [9]:
!pip install -q transformers datasets sentencepiece accelerate rouge-score


In [10]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import pandas as pd
import numpy as np
import torch
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import Dataset
from rouge_score import rouge_scorer
import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [12]:
TRAIN_PATH = "/content/drive/MyDrive/train.csv"
VAL_PATH   = "/content/drive/MyDrive/validation.csv"
TEST_PATH  = "/content/drive/MyDrive/test.csv"

In [13]:
def load_and_preprocess_data(train_path, val_path, test_path):
    print("Loading datasets...")

    train_df = pd.read_csv(train_path)
    val_df   = pd.read_csv(val_path)
    test_df  = pd.read_csv(test_path)

    # Remove missing values
    train_df = train_df.dropna(subset=["article", "highlights"]).reset_index(drop=True)
    val_df   = val_df.dropna(subset=["article", "highlights"]).reset_index(drop=True)
    test_df  = test_df.dropna(subset=["article", "highlights"]).reset_index(drop=True)

    # Add T5 prefix
    for df in (train_df, val_df, test_df):
        df["input_text"]  = "summarize: " + df["article"].astype(str)
        df["target_text"] = df["highlights"].astype(str)

    print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")
    return train_df, val_df, test_df

train_df, val_df, test_df = load_and_preprocess_data(TRAIN_PATH, VAL_PATH, TEST_PATH)


Loading datasets...
Train: 287113, Val: 13368, Test: 11490


In [1]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

def create_datasets(df, tokenizer, max_input_len=512, max_target_len=128):

    def tokenize(batch):
        model_inputs = tokenizer(
            batch["input_text"],
            max_length=max_input_len,
            truncation=True,
            padding="max_length"
        )
        labels = tokenizer(
            batch["target_text"],
            max_length=max_target_len,
            truncation=True,
            padding="max_length"
        )
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    dataset = Dataset.from_pandas(df[["input_text", "target_text"]].reset_index(drop=True))
    dataset = dataset.map(tokenize, batched=True, remove_columns=["input_text", "target_text"])
    return dataset

train_dataset = create_datasets(train_df, tokenizer)
val_dataset   = create_datasets(val_df, tokenizer)
test_dataset  = create_datasets(test_df, tokenizer)


NameError: name 'T5Tokenizer' is not defined

In [ ]:
def train_model(train_dataset, val_dataset, tokenizer):
    model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)

    data_collator = DataCollatorForSeq2Seq(tokenizer, model)

    training_args = TrainingArguments(
        output_dir="./t5_summarizer",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_steps=100,
        learning_rate=3e-4,
        per_device_train_batch_size=2,   # safe for Colab
        per_device_eval_batch_size=2,
        num_train_epochs=3,
        weight_decay=0.01,
        fp16=torch.cuda.is_available(),
        load_best_model_at_end=True,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator
    )

    trainer.train()
    trainer.save_model("./t5_summarizer")
    tokenizer.save_pretrained("./t5_summarizer")

    return trainer, model

trainer, model = train_model(train_dataset, val_dataset, tokenizer)


In [ ]:
def generate_summary(text, model, tokenizer, max_len=128):
    inputs = tokenizer(
        "summarize: " + text,
        return_tensors="pt",
        max_length=512,
        truncation=True
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        output = model.generate(
            inputs["input_ids"],
            max_length=max_len,
            num_beams=4,
            no_repeat_ngram_size=2,
            early_stopping=True
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
def evaluate_rouge(test_df, model, tokenizer, samples=50):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

    test_sample = test_df.sample(samples, random_state=42)

    r1 = []; r2 = []; rl = []

    for _, row in test_sample.iterrows():
        generated = generate_summary(row["article"], model, tokenizer)
        reference = row["highlights"]
        scores = scorer.score(reference, generated)

        r1.append(scores["rouge1"].fmeasure)
        r2.append(scores["rouge2"].fmeasure)
        rl.append(scores["rougeL"].fmeasure)

    print("\nROUGE SCORES")
    print("ROUGE-1:", np.mean(r1))
    print("ROUGE-2:", np.mean(r2))
    print("ROUGE-L:", np.mean(rl))

    return {
        "ROUGE-1": np.mean(r1),
        "ROUGE-2": np.mean(r2),
        "ROUGE-L": np.mean(rl)
    }

rouge_scores = evaluate_rouge(test_df, model, tokenizer)


In [ ]:
def show_examples(test_df, model, tokenizer, num=3):
    samples = test_df.sample(num, random_state=42)

    for i, row in samples.iterrows():
        print("\n==============================")
        print("ORIGINAL ARTICLE:")
        print(row["article"][:350], "...\n")

        print("REFERENCE SUMMARY:")
        print(row["highlights"], "\n")

        print("GENERATED SUMMARY:")
        print(generate_summary(row["article"], model, tokenizer))
        print("==============================\n")

show_examples(test_df, model, tokenizer, num=3)
